# <a id='toc1_'></a>[Amazon тест](#toc0_)

In [ ]:

import header
from header import  start_supplier, Product, ProductFields

supplier_prefix = 'amazon'
s = start_supplier(supplier_prefix)
""" s - на протяжении всего кода означает класс `Supplier` """

print(" Можно продолжать ")

<p><font color="#074B35">Сценарий из тестового файла `dict_scenarios.py` </font></p>

In [3]:
from dict_scenarios import scenario
s.current_scenario = scenario['Murano Glass']
l = s.locators.get('product')
d = s.driver
_ = d.execute_locator

<p> Сценарий из файла сценариев  </p>

In [4]:
print(s.__doc__)

  Supplier class
    Класс поставщика. Исполняет сценарии

        Named Args:
        `supplier_prefix` `str`  :  supplier_prefix s.e. 'aliexpress'
                    Table of suppliers: https://docs.google.com/spreadsheets/d/14f0PyQa32pur-sW2MBvA5faIVghnsA0hWClYoKpkFBQ/edit?usp=sharing
        `land` `str`  :   language in ISO 639-1
        `supplier_id` `int`  :  supplier id     Table of suppliers:
    https://docs.google.com/spreadsheets/d/14f0PyQa32pur-sW2MBvA5faIVghnsA0hWClYoKpkFBQ/edit?usp=sharing
    


<h3><i>Файл стаскивается с моего сервера, a нe с Амазон. </i></h3>

In [4]:
test_url_1 = r"https://dev.davidka.net/hypotez/amazon/amazon_test_1.htm"
test_url_2 = r"https://dev.davidka.net/hypotez/amazon/amazon_test_2.htm"

<h3><i>Файл стаскивается с Амазон. </i></h3>

In [5]:
test_url_3 = r"https://www.amazon.com/Apple-Watch-44mm-Space-Aluminum/dp/B07PS9D7GW/ref=sr_1_1?qid=1685717078&refinements=p_n_is_free_shipping%3A10236242011%2Cp_89%3AApple&rnid=2407755011&s=electronics&sr=1-1"

In [5]:
test_url_4 = r"https://www.amazon.com/C%C3%A1-dOro-Hippie-Colored-Murano-Style/dp/B09N53XSQB/ref=sr_1_1_sspa?crid=24Q0ZZYVNOQMP&keywords=Art+Deco+murano+glass&qid=1687277030&sprefix=art+deco+murano+glass%2Caps%2C230&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1"

In [6]:
d.get_url(test_url_4)

True

_______________________________________________
## <a id='toc1_3_'></a>[Поля товара](#toc0_)

In [7]:
from src.tools import StringNormalizer as StringNormalizer
def grab_product_page(s):
    """ 
        Собираю информацию со страницы товара
        Attrs:
            p (Product)
            d (Driver)
        @returns
            f (ProductFields) с заполненными полями, else False
    """
    d = s.driver
    l = s.locators.get('product')
    f = ProductFields()
    _ = d.execute_locator



    """ ID,ASIN,SKU,SUPPLIER SKU """
    

    def _set_defaults() -> bool:
        """ Set defaults for product of supplier
        Attrs:
    
            
        @returns
    	"""
        f.active = 1
        f.active = 1
        f.on_sale = 0
        f.min_qty = 1
        f.low_stock_level = 0
        f.low_stock_threshold = ''
        f.show_price = 1
        f.show_condition = 1
        f.aviable_online_only = 0
        f.advanced_stock_management = 0
        
        ...

    _set_defaults()

    ASIN = _(l['ASIN'])
    f.id_product = None
    f.reference = f'{s.supplier_id}-{ASIN}'
    f.supplier_reference = ASIN
    f.price = StringNormalizer.normalize_price(_(l['price'])[0])
    f.name = _(l['name'])[0]
    #f.summary = _(l['summary'])[0]
    f.id_supplier = s.supplier_id
    f.id_category_default = list(s.current_scenario['presta_categories']['default_category'].keys())[0]
    return f


## Переопределяю локаторы для товара по test_url_4

In [13]:
l['price']['selector'] = \
"//div[contains(@data-csa-c-asin, '$_(driver.current_url.split(f'''/''')[-2])_$') and contains(@id, 'corePriceDisplay')]//span[1]"

In [14]:
ASIN = _(l['ASIN'])

In [15]:
product_reference = f"{s.supplier_id}-{ASIN}"
check = Product.check_if_product_in_presta_db(product_reference)

In [16]:
print(check)

45


In [17]:
product_fields: ProductFields = grab_product_page(s)

In [18]:
raw_price = _(l['price'])[0]


In [19]:
print(raw_price)

$168.00
$168
.
00


In [20]:
raw_price = str(raw_price).split('\n')[0]

In [21]:
print(raw_price)

$168.00


In [22]:
price = StringNormalizer.normalize_price(raw_price)

In [23]:
jprint(price)
print(type(price))

168.0
<class 'float'>


In [30]:
product_fields.wholesale_price = product_fields.price = str(price)
product_fields.name = product_fields.name.encode('utf-8')
product_dict: dict = {}
product_dict['product']: dict = dict(product_fields.fields)
#product_dict['product']['wholesale_price'] = product_dict['product']['price'] = float(product_dict['product']['wholesale_price'] )

In [34]:
print(product_dict)

{'product': {'active': 1, 'on_sale': 0, 'low_stock_threshold': '', 'show_price': 1, 'show_condition': 1, 'advanced_stock_management': 0, 'id_product': None, 'reference': '2800-B09N53XSQB', 'supplier_reference': 'B09N53XSQB', 'name': b"C\xc3\xa1 d'Oro Small Glass Vase Hippie Colored Canes Hand Blown Murano-Style Art Glass - Model Oval", 'id_supplier': '2800', 'id_category_default': '11209', 'wholesale_price': '168.0', 'price': '168.0'}}


In [35]:
from src.endpoints.PrestaShop import Product as PrestaProduct

In [36]:
PrestaProduct.add(product_dict)

{'PrestaShop': {'product': {'id': '51', 'id_manufacturer': '', 'id_supplier': '2800', 'id_category_default': '11209', 'new': '', 'cache_default_attribute': '', 'id_default_image': {'attrs': {'notFilterable': 'true'}, 'value': ''}, 'id_default_combination': {'attrs': {'notFilterable': 'true'}, 'value': '0'}, 'id_tax': '', 'position_in_category': {'attrs': {'notFilterable': 'true'}, 'value': ''}, 'manufacturer_name': {'attrs': {'notFilterable': 'true'}, 'value': ''}, 'quantity': {'attrs': {'notFilterable': 'true'}, 'value': ''}, 'type': {'attrs': {'notFilterable': 'true'}, 'value': 'simple'}, 'id_shop_default': '1', 'reference': '2800-B09N53XSQB', 'supplier_reference': 'B09N53XSQB', 'location': '', 'width': '', 'height': '', 'depth': '', 'weight': '', 'quantity_discount': '', 'ean13': '', 'isbn': '', 'upc': '', 'mpn': '', 'cache_is_pack': '', 'cache_has_attachments': '', 'is_virtual': '', 'state': '', 'additional_delivery_times': '', 'delivery_in_stock': {'language': [{'attrs': {'id': '1

In [ ]:

f.ean13= _(l['ean13']) or 0
print(f"""'ean13' = {f.ean13}""")

f.isbn= _(l['isbn']) or 0
print(f"""'ean13' = {f.isbn}""")

f.upc= _(l['upc']) or 0
print(f"""'upc' = {f.upc}""")

f.mpn= _(l['mpn']) or 0
print(f"""'mpn' = {f.mpn}""")

f.ecotax= _(l['ecotax']) or 0
print(f"""'ecotax' = {f.ecotax}""")

f.quantity= _(l['quantity']) or 1
print(f"""'quantity' = {f.quantity}""")

f.minimal_quantity= _(l['minimal_quantity']) or 0
print(f"""'minimal_quantity' = {f.minimal_quantity}""")
        

In [ ]:

f.wholesale_price= _(l['wholesale_price']) or 0
print(f"""'wholesale_price' = {f.wholesale_price}""")
f.unity= _(l['unity'])
print(f"""'unity' = {f.unity}""")
f.unit_price_ratio= _(l['unit_price_ratio'])
print(f"""'unit_price_ratio' = {f.unit_price_ratio}""")
f.additional_shipping_cost= _(l['additional_shipping_cost'])
print(f"""'additional_shipping_cost' = {f.additional_shipping_cost}""")
f.reference= _(l['additional_shipping_cost'])
print(f"""'reference' = {f.reference}""")
f.supplier_reference= _(l['supplier_reference'])
print(f"""'supplier_reference' = {f.supplier_reference}""")
f.location= _(l['location'])
print(f"""'location' = {f.location}""")
f.width= _(l['width'])
print(f"""'width' = {f.width}""")
f.height= _(l['height'])
print(f"""'height' = {f.height}""")
f.depth= _(l['depth'])
print(f"""'depth' = {f.depth}""")
f.weight= _(l['weight'])
print(f"""'weight' = {f.weight}""")
f.out_of_stock= _(l['out_of_stock'])
print(f"""'out_of_stock' = {f.out_of_stock}""")
f.additional_delivery_times= _(l['additional_delivery_times'])
print(f"""'additional_delivery_times' = {f.additional_delivery_times}""")
f.quantity_discount= _(l['quantity_discount'])
print(f"""'quantity_discount' = {f.quantity_discount}""")

In [ ]:
description= _(l['description'])
description_short= _(l['description_short'])
link_rewrite= f.link_rewrite
meta_description= _(l['meta_description'])
meta_keywords= _(l['meta_keywords'])
meta_title= _(l['meta_title'])
name= _(l['name'])
available_now= f.available_now
available_later= f.available_later        
delivery_in_stock= _(l['delivery_in_stock'])
delivery_out_stock= _(l['delivery_out_stock'])
affiliate_short_link= _(l['affiliate_short_link'])
affiliate_text= _(l['affiliate_text'])
affiliate_summary= _(l['affiliate_summary'])
affiliate_summary_2= _(l['affiliate_summary_2'])

In [ ]:
field_price_tax_exluded
field_price_tax_included
field_tax_rule_id

In [ ]:
field_cost_price

In [ ]:
field_on_sale
field_discount_amount
field_discount_percent
field_discount_from
field_discount_to

In [ ]:

field_sku
field_supplier_sku
field_supplier_id
field_brand

In [ ]:
s.run_scenario(test_scenario)

In [ ]:
print(s)